In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import preprocessing
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import f_regression
from matplotlib import pyplot
import tensorflow
tensorflow.random.set_seed(1)
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import xgboost as xg 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 
from catboost import CatBoostRegressor
from xgboost import XGBClassifier

In [ ]:
df_train=pd.read_csv(os.path.join(dirname,'train.csv'))
df_test=pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv')

In [ ]:
df_train


In [ ]:
df_train.info()

In [ ]:
df_train['Survived'].unique()

In [ ]:
df_train['Survived'].value_counts()

In [ ]:
for col in df_train.columns:
    print(df_train[col].unique())
    print(len(df_train[col].unique()))
    #(df_train[col].value_counts)

In [ ]:
from collections import Counter
counter = Counter(list(df_train['Cabin']))

In [ ]:
#counter

In [ ]:
for col in df_train.columns:
    print(col,":",df_train[col].isnull().values.any())

In [ ]:
df_train['Fare'].fillna(value=0,inplace=True)
df_train['Ticket'].fillna(value='NOA',inplace=True)
df_train['Cabin'].fillna(value='NoCabin',inplace=True)
df_train['Embarked'].fillna(value='unknown',inplace=True)
df_train['Age'].fillna(value=round(df_train['Age'].mean()),inplace=True)

In [ ]:
df_train

In [ ]:
#One hot Encoding
#categorical fields Pclass,Sex,Embarked,Cabin
df_train=pd.concat([df_train,pd.get_dummies(df_train.Pclass,prefix='Pclass')],axis=1)
df_train=df_train.drop(['Pclass','Pclass_1'],axis=1)
df_train=pd.concat([df_train,pd.get_dummies(df_train.Sex,prefix='Sex')],axis=1)
df_train=df_train.drop(['Sex','Sex_male'],axis=1)
df_train=pd.concat([df_train,pd.get_dummies(df_train.Embarked,prefix='Embarked')],axis=1)
df_train=df_train.drop(['Embarked','Embarked_S'],axis=1)
#df_train=pd.concat([df_train,pd.get_dummies(df_train.Cabin,prefix='Cabin')],axis=1)
#df_train=df_train.drop(['Cabin','Cabin_NoCabin'],axis=1)

In [ ]:
df_train

In [ ]:
X=df_train.drop(['PassengerId','Survived','Name','Ticket','Cabin'],axis=1).values

In [ ]:

from sklearn.preprocessing import MinMaxScaler


In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
y=df_train['Survived'].values

TEST DATA

In [ ]:
for col in df_test.columns:
    print(col,":",df_test[col].isnull().values.any())

In [ ]:
### Preprocessing Test Data
df_test['Fare'].fillna(value=0,inplace=True)
df_test['Ticket'].fillna(value='NOA',inplace=True)
df_test['Cabin'].fillna(value='NoCabin',inplace=True)
df_test['Embarked'].fillna(value='unknown',inplace=True)
df_test['Age'].fillna(value=round(df_train['Age'].mean()),inplace=True)

#One hot Encoding
#categorical fields Pclass,Sex,Embarked,Cabin
df_test=pd.concat([df_test,pd.get_dummies(df_test.Pclass,prefix='Pclass')],axis=1)
df_test=df_test.drop(['Pclass','Pclass_1'],axis=1)
df_test=pd.concat([df_test,pd.get_dummies(df_test.Sex,prefix='Sex')],axis=1)
df_test=df_test.drop(['Sex','Sex_male'],axis=1)
df_test=pd.concat([df_test,pd.get_dummies(df_test.Embarked,prefix='Embarked')],axis=1)
df_test=df_test.drop(['Embarked','Embarked_S'],axis=1)
df_train=pd.concat([df_train,pd.get_dummies(df_train.Cabin,prefix='Cabin')],axis=1)
df_train=df_train.drop(['Cabin','Cabin_NoCabin'],axis=1)

In [ ]:
p_id=df_test['PassengerId']

In [ ]:
X_test=df_test.drop(['PassengerId','Name','Ticket','Cabin'],axis=1).values

In [ ]:
X_test_scaled=scaler.transform(X_test)

In [ ]:
def select_features(X_train, y_train, X_test):
    # configure to select all features
    fs = SelectKBest(score_func=f_regression, k=6)
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

In [ ]:
X_test_scaled.shape,X_scaled.shape

In [ ]:
X_train_fs, X_test_fs, fs = select_features(X_scaled, y, X_test_scaled)

for i in range(len(fs.scores_)):
    print('Feature %d: %f' % (i, fs.scores_[i]))

pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
pyplot.show()

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier()
model.fit(X_train_fs, y) 

In [ ]:
y_pred = model.predict(X_test_fs)
predictions = [round(value) for value in y_pred]

In [ ]:
csvlist=[]
for i in range(len(predictions)):
    csvlist.append([p_id[i],predictions[i]])

In [ ]:
Output=pd.DataFrame(csvlist,columns=['PassengerId','Survived'])
Output.to_csv('output_XGB.csv',index=False)

ELASTIC NET

In [ ]:
 from sklearn.linear_model import ElasticNet

In [ ]:
reg = ElasticNet(random_state=0)
reg.fit(X_train_fs, y)

In [ ]:
y_pred = reg.predict(X_test_fs)
predictions = [round(value) for value in y_pred]

In [ ]:
csvlist=[]
for i in range(len(predictions)):
    csvlist.append([p_id[i],predictions[i]])

In [ ]:
Output=pd.DataFrame(csvlist,columns=['PassengerId','Survived'])
Output.to_csv('output_ElasticNet.csv',index=False)